In [ ]:
import face_recognition
import cv2

#for reference by webcam
capturing_video = cv2.VideoCapture(0)

# Load a picture and learn how to recognize it.
modi_image = face_recognition.load_image_file("./modi.jpg")
modi_face_encoding = face_recognition.face_encodings(modi_image)[0]

# Load a second picture and learn how to recognize it.
harsh_image = face_recognition.load_image_file("./harsh.jpg")
harsh_face_encoding = face_recognition.face_encodings(harsh_image)[0]

# Create array which store all face encoding and their name
known_face_encodings = [
    modi_face_encoding,
    harsh_face_encoding
]

known_face_names = [
    "Narendra Modi",
    "Harsh Tyagi"
]


face_loc = []
face_encodings = []
face_names = []
this_frame = True

while True:
    #capture a frame
    ret, frame = capturing_video.read()

    # Resize frame
    resize_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert image from BGR color to RGB color
    rgb_resize_frame = resize_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if this_frame:
        
        # Find all the faces and face encodings in the current frame of video 
        face_loc = face_recognition.face_loc(rgb_resize_frame)
        
        face_encodings = face_recognition.face_encodings(rgb_resize_frame, face_loc)

        face_names = []
        for face_encoding in face_encodings:

            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]

            face_names.append(name)
            

    this_frame = not this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_loc, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


capturing_video.release()
cv2.destroyAllWindows()